In [1]:
import napari
import dask.array as da

## Zarr

In [4]:
%%time
da_stack_gfp = da.from_zarr(f"/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/zarr/tzxy_stack_ch{1}.zarr")
da_stack_rfp = da.from_zarr(f"/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/zarr/tzxy_stack_ch{2}.zarr")
v = napari.Viewer()
v.add_image(da_stack_gfp, name="gfp", contrast_limits = [0,2352], blending = 'additive', colormap= 'green')# colormap = 'g')
v.add_image(da_stack_rfp, name="rfp", contrast_limits = [103,164], blending = 'additive', colormap = 'magenta')

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


CPU times: user 1.02 s, sys: 559 ms, total: 1.58 s
Wall time: 1.12 s


<Image layer 'rfp' at 0x7fa6b81822e0>

In [5]:
da_stack_gfp

,Array,Chunk
Bytes,15.33 GiB,8.90 MiB
Shape,"(75, 3, 6048, 6048)","(1, 1, 2160, 2160)"
Count,2026 Tasks,2025 Chunks
Type,uint16,numpy.ndarray


## TIFF

In [6]:
import dask.array as da
import dask
import glob
from skimage.io import imread

In [7]:
image_fns = glob.glob('/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=proj-lm-gutierrezm/output_data/September_22_2021/Row_3_Col_3_trimmed/MIPs/Time*.TIFF')

In [8]:
%%time
da_tiff_stack_gfp = [dask.delayed(imread(fn)) for fn in image_fns]

CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 14.5 µs


In [9]:
da_tiff_stack_gfp

[]

In [32]:
%%time 
da_tiff_stack_gfp = [da.from_delayed(x, shape = (6048,6048), dtype= "u2") for x in da_tiff_stack_gfp]

CPU times: user 1.02 ms, sys: 0 ns, total: 1.02 ms
Wall time: 1.06 ms


In [35]:
da_tiff_stack_gfp[0]

dask.array<from-value, shape=(6048, 6048), dtype=uint16, chunksize=(6048, 6048), chunktype=numpy.ndarray>

### Testing with raw images... but at the moment these raw images are untiled so what is the point at this stage if I already know using dask.delayed will take longer than just dropping a zarr

# Reconfiguring the order of the zarr dask array

In [36]:
da_stack_gfp = da.from_zarr(f"data/zt_stack_ch{1}.zarr")
da_stack_rfp = da.from_zarr(f"data/zt_stack_ch{2}.zarr")

In [38]:
da_stack_gfp.shape

(3, 75, 6048, 6048)

#### shape needs to be tczxy ie. (75, 2, 3, 6048, 6048)

In [43]:
reshape = da.reshape(da_stack_gfp, shape = (225, 6048, 6048), merge_chunks=False)
reshape

dask.array<reshape, shape=(225, 6048, 6048), dtype=uint16, chunksize=(1, 2160, 2160), chunktype=numpy.ndarray>

In [45]:
reshape = da.reshape(reshape, shape = (75, 3, 6048, 6048))
reshape

,Array,Chunk
Bytes,15.33 GiB,26.70 MiB
Shape,"(75, 3, 6048, 6048)","(1, 3, 2160, 2160)"
Count,5401 Tasks,675 Chunks
Type,uint16,numpy.ndarray


In [46]:
v = napari.Viewer()
v.add_image(reshape, name="gfp", contrast_limits = [0,2352], blending = 'additive', colormap= 'green')# colormap = 'g')

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'gfp' at 0x7fd935a47b80>